# Исследование надежности заемщиков


Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Данные имеют следующие столбцы
*  children — количество детей в семье
*  days_employed — общий трудовой стаж в днях
*  dob_years — возраст клиента в годах
*  education — уровень образования клиента
*  education_id — идентификатор уровня образования
*  family_status — семейное положение
*  family_status_id — идентификатор семейного положения
*  gender — пол клиента
*  income_type — тип занятости
*  debt — имел ли задолженность по возврату кредитов
*  total_income — ежемесячный доход
*  purpose — цель получения кредита


## План работы над проектом
1. Провести анализ пропусков, изменения данных и повторения значений.
2. Определить, какие некорректные строки требуется сохранить, а какие можно удалить.
3. Заполнить значения в пропущенных ячейках таблицы
4. Удалить дубликаты.
5. Вывести категории для проведения гипотез.
6. Составить выводы по каждому вопросу.
7. Составить общий вывод.

Во второй части проекта вы выполните шаги 3 и 4. Их вручную проверит ревьюер.
Чтобы вам не пришлось писать код заново для шагов 1 и 2, мы добавили авторские решения в ячейки с кодом. 



## Откройте таблицу и изучите общую информацию о данных

Подключение необходимы библиотек

In [2]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
import copy as c
import numpy as np

**Задание 1. Импортируйте библиотеку pandas. Считайте данные из csv-файла в датафрейм и сохраните в переменную `data`. Путь к файлу:**

`/datasets/data.csv`

In [1]:
data = pd.read_csv('/datasets/data.csv')

**Задание 2. Выведите первые 20 строчек датафрейма `data` на экран.**

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Задание 3. Выведите основную информацию о датафрейме с помощью метода `info()`.**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**Задание 4. Выведите количество пропущенных значений для каждого столбца. Используйте комбинацию двух методов.**

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Задание 5. В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце вы обработаете на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**Задание 6. В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработайте значения в этом столбце: замените все отрицательные значения положительными с помощью метода `abs()`.**

In [6]:
data['days_employed'] = data['days_employed'].abs()

**Задание 7. Для каждого типа занятости выведите медианное значение трудового стажа `days_employed` в днях.**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

**Задание 8. Выведите перечень уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**Задание 9. В столбце `children` есть два аномальных значения. Удалите строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Задание 10. Ещё раз выведите перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

**Задание 11. Заполните пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Задание 12. Убедитесь, что все пропуски заполнены. Проверьте себя и ещё раз выведите количество пропущенных значений для каждого столбца с помощью двух методов.**

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Задание 13. Замените вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Задание 14. Обработайте неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к нижнему регистру. Проверьте остальные столбцы.**

In [14]:
data['education'] = data['education'].str.lower()

**Задание 15. Выведите на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалите их.**

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

**Задание 16. На основании диапазонов, указанных ниже, создайте в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используйте собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Задание 17. Выведите на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Задание 18. Создайте функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используйте собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучите данные в столбце `purpose` и определите, какие подстроки помогут вам правильно определить категорию.**

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
data_pivot = data.pivot_table(index = ['children'], values = 'debt', aggfunc = 'sum')# Создадим сводную таблицу для обощения данных и их представления
print(data_pivot)# выводим сводную таблицу на экран

          debt
children      
0         1063
1          444
2          194
3           27
4            4
5            0


In [23]:
credit = data.groupby('children')['children'].count()#общее количество клиентов для каждой группы
print(credit)# выведим на экран количество клиентов

children
0    14091
1     4808
2     2052
3      330
4       41
5        9
Name: children, dtype: int64


In [24]:
debtor = data.groupby('children')['debt'].sum()#общее количество просроченных кредитов для семей с детьми и без

In [25]:
conversion = (debtor/credit)*100#расчитаем конверсию для каждой группы клиентов
print(conversion)

children
0    7.543822
1    9.234609
2    9.454191
3    8.181818
4    9.756098
5    0.000000
dtype: float64


In [26]:
data.pivot_table(index = 'children', values = 'debt', aggfunc = ['count', 'sum', 'mean'])\
.sort_values(by = ('mean', 'debt'), ascending = False)#построим сводную таблицу для анализа данных

,count,sum,mean
,debt,debt,debt
children,,,
4,41,4,0.097561
2,2052,194,0.094542
1,4808,444,0.092346
3,330,27,0.081818
0,14091,1063,0.075438
5,9,0,0.000000


**Вывод:** Полученные данные говорят о том, что наличие детей влияет на процент возврата кредита.Так например, клиенты у которых нет детей, самый низкий процент конверсии 7%, это значит что клиенты без детей больше вероятны оплачивать кредиты в срок, ведь самая большая доля клиентов, это клиенты без детей, которая составляет 66%. Клиенты с одним, двумя конверсия почти одинаковая - 9%, самой высокой в данной группе, значит непосредственно наличие детей прямо влияет на возврат кредита в срок, и их доля составляет 22% и 9% соответственно. У клиентов с тремя детьми конверсия составляет 8%, их доля в общем количестве семей, составила 1,5%. Для семей с четверьмя детьми конверсия составила 9%, их доля сосавила 0,2%. У семей с пятью детьми, конверсия 0%, и их доля составила 0,04%. Не стоит использовать данные клиентов с четырьмя и петью детьми для выявления зависисмости возврата кредита в срок. 

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [27]:
data_pivot = data.pivot_table(index = ['family_status'], values = 'debt', aggfunc = 'sum')# Создадим сводную таблицу для обощения данных и их представления
print(data_pivot)# выводим сводную таблицу на экран

                       debt
family_status              
Не женат / не замужем   273
в разводе                84
вдовец / вдова           63
гражданский брак        385
женат / замужем         927


In [28]:
number_of_families = data.groupby('family_status')['family_status'].count()#общее количество семей для каждой группы
print(number_of_families)# выведим на экран количество семей

family_status
Не женат / не замужем     2796
в разводе                 1189
вдовец / вдова             951
гражданский брак          4134
женат / замужем          12261
Name: family_status, dtype: int64


In [29]:
debtor_family = data.groupby('family_status')['debt'].sum()#сгруппируем общее количество семей имевших задолженность по кредиту

In [30]:
conversion = (debtor_family/number_of_families)*100#расчитаем конверсию
print(conversion)

family_status
Не женат / не замужем    9.763948
в разводе                7.064760
вдовец / вдова           6.624606
гражданский брак         9.313014
женат / замужем          7.560558
dtype: float64


In [31]:
data.pivot_table(index = 'family_status', values = 'debt', aggfunc = ['count', 'sum', 'mean'])\
.sort_values(by = ('mean', 'debt'), ascending = False)#построим сводную таблицу для анализа данных

,count,sum,mean
,debt,debt,debt
family_status,,,
Не женат / не замужем,2796,273,0.097639
гражданский брак,4134,385,0.093130
женат / замужем,12261,927,0.075606
в разводе,1189,84,0.070648
вдовец / вдова,951,63,0.066246


**Вывод:** На основе полученных данных можно сделать следующий вывод. Cамый высокий процент конверсии около 9% у сдледующих категорий семей "не женат/не замужем, гражданский брак". Более отвественные клиенты следующиx категорий граждан: "вдова/вдовец", "в разводе", "женат/замужем", здесь соответсвенно конверсия составила 6%, 7%, 7%. Таким образом, мы видим что клиенты имеющие различное семейное положение и видимо их образ жизни влияют на ответсвенность по возврату кредита в срок. Клиенты, побывавшие в браке, реже допускают просрочки.  

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [32]:
data_pivot = data.pivot_table(index = ['total_income_category'], values = 'debt', aggfunc = 'sum')# Создадим сводную таблицу для обощения данных и их представления
print(data_pivot)# выводим сводную таблицу на экран

                       debt
total_income_category      
A                         2
B                       354
C                      1353
D                        21
E                         2


In [33]:
number_income_category = data.groupby('total_income_category')['total_income_category'].count()#общее количество клиентов для каждой категории
print(number_income_category)# выведим на экран количество клиентов по категориям

total_income_category
A       25
B     5014
C    15921
D      349
E       22
Name: total_income_category, dtype: int64


In [34]:
debtor_total_income = data.groupby('total_income_category')['debt'].sum()#сгруппируем общее количество клиентов имевших задолженность по кредиту

In [35]:
conversion = (debtor_total_income/number_income_category)*100#расчитаем конверсию для каждой категории клиентов
print(conversion)

total_income_category
A    8.000000
B    7.060231
C    8.498210
D    6.017192
E    9.090909
dtype: float64


In [36]:
data.pivot_table(index = 'total_income_category', values = 'debt', aggfunc = ['count', 'sum', 'mean'])\
.sort_values(by = ('mean', 'debt'), ascending = False)# построим сводную таблицу для анализа данных

,count,sum,mean
,debt,debt,debt
total_income_category,,,
E,22,2,0.090909
C,15921,1353,0.084982
A,25,2,0.080000
B,5014,354,0.070602
D,349,21,0.060172


**Вывод:** Ранее мы категоризировали всех лиц по уровню дохода следующим образом:

0–30000 — 'E';

30001–50000 — 'D';

50001–200000 — 'C';

200001–1000000 — 'B';

1000001 и выше — 'A'.

По полученным данным можно сделать вывод, что клиенты с доходами "В" (200001-1000000 руб.) совершают реже просроченных платежей - 7% от общей доли категорий клиентов 23%. Самый высокий процент просроченных кредитов у группы клиентов с доходами "С" (50001-200000 руб.), здесь 8 % не возвратов кредитов, причем доля "С" категории клиентов составляет 74%, что существенно влияет значение зависимоти между уровнем дохода и возвратом кредита. Категории "A, D, E" в значение зависимости не берем, так как здесь низкая доля категорий клиентов в общей сумме клиентов. 

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [37]:
data_pivot = data.pivot_table(index = ['purpose_category'], values = 'debt', aggfunc = 'sum')# Создадим сводную таблицу для обощения данных и их представления
print(data_pivot)# выводим сводную таблицу на экран# Ваш код будет здесь. Вы можете создавать новые ячейки.

                          debt
purpose_category              
операции с автомобилем     400
операции с недвижимостью   780
получение образования      369
проведение свадьбы         183


In [38]:
number_purpose_category = data.groupby('purpose_category')['purpose_category'].count()#общее количество клиентов для каждой группы
print(number_purpose_category)# выведим на экран количество клиентов для каждой цели кредита

purpose_category
операции с автомобилем       4279
операции с недвижимостью    10751
получение образования        3988
проведение свадьбы           2313
Name: purpose_category, dtype: int64


In [39]:
debtor_purpose = data.groupby('purpose_category')['debt'].sum()#сгруппируем общее количество клиентов имевших задолженность по кредиту

In [40]:
conversion = (debtor_purpose/number_purpose_category)*100#расчитаем конверсию
print(conversion)

purpose_category
операции с автомобилем      9.347978
операции с недвижимостью    7.255139
получение образования       9.252758
проведение свадьбы          7.911803
dtype: float64


In [41]:
data.pivot_table(index = 'purpose_category', values = 'debt', aggfunc = ['count', 'sum', 'mean'])\
.sort_values(by = ('mean', 'debt'), ascending = False)# построим сводную таблицу для анализа данных

,count,sum,mean
,debt,debt,debt
purpose_category,,,
операции с автомобилем,4279,400,0.093480
получение образования,3988,369,0.092528
проведение свадьбы,2313,183,0.079118
операции с недвижимостью,10751,780,0.072551


**Вывод:** По полученным данным мы видим, что для получения кредита на подкатегорию "операции с автомобилем" и "получения образования" выходит самый высокий процент просроченных кредитов, а для подкатегории "операции с недвижимостью" и "проведение свадьбы" чуть меньше, 7%, но меньше вероятности просроченных кредитов относительно целей выше.

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* Пропуски появляются по-разному. Возможно, что клиенты сами умышленно не хотят заполнять данные, возможно они их заполняют не верно, а возможно автоматизированная система дала сбой.

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* Медианым значением лучше заполнить пропуски, если например сильно разняца данные, чтобы не брать выдающиеся данные, ищут медианну, например в случае заработной платы, или еще каких-то входных данных, имеющих смысл для расета значений. 

### Шаг 4: общий вывод.

Для построения модели кредитного скоринга стоит учитывать количество семей без детей, а также детьми - одного, два и трое. Стоит обратить внимание на семейное положение, например вдовец/вдова, где самый низкий процент возврата кредита, уровень дохода клиентов "В" (200001-1000000 руб.) рублей, где самый низкий процент возврата кредита и "С" (50001-200000 руб.), где самый высокий процент не возвратов кредитов. Так же стоит учитывать цели кредита, например на операции с автомобилем и получения образования, самый высокий процент задолжников по кредитам. Следовательно, для каждой группы нужно расчитать скорингонговую  модель, и учитывать все нюансы.